In [43]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from cloudmesh.common.StopWatch import StopWatch
from pandas.plotting import register_matplotlib_converters
from sklearn.metrics import mean_squared_error
import math
import datetime as dt
from configuration import read_config
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [44]:
config = read_config("config-gregor.yaml")

# config.debug = False
print (config)
config.debug=False

epochs=[2, 10,50, 75, 100, 125, 150, 175, 200, 225]
repeat=5

{'debug': True, 'user': 'gregor', 'node': '5090X-RTX3090', 'experiments.epoch': [1, 2, 10, 20, 30], 'experiments.gpu': ['K80', 'A100', 'V100', 'P100'], 'sep': '.'}


In [45]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, time_steps=10):
        self.X = X
        self.y = y
        self.time_steps = time_steps

    def __len__(self):
        return len(self.X) - self.time_steps

    def __getitem__(self, index):
        X = self.X.iloc[index:(index + self.time_steps)].values
        y = self.y.iloc[index + self.time_steps]
        return torch.tensor(X).float(), torch.tensor(y).float()

In [46]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

In [47]:
cryptos = ["EOS-USD", "BTC-USD", "ETH-USD", "DOGE-USD"]
crypto = "EOS-USD"
ticker = yf.Ticker(crypto)
data = ticker.history(period="max", interval="1d")
# print(data)
# Sort the dataframe according to the date
data.sort_values('Date', inplace=True, ascending=True)

# Get Close data
df = data[['Close']].copy()
# Split data into train and test
train, test = df.iloc[0:-200], df.iloc[-200:len(df)]

train_max = train.max()
train_min = train.min()

# Normalize the dataframes
train = (train - train_min) / (train_max - train_min)
test = (test - train_min) / (train_max - train_min)

In [48]:
time_steps = 10
train_dataset = TimeSeriesDataset(train, train.Close, time_steps)
test_dataset = TimeSeriesDataset(test, test.Close, time_steps)

In [49]:
# Define the batch size and shuffle option
batch_size = 32
shuffle_train = False
shuffle_test = False

# Create DataLoaders for train and test sets
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle_train, drop_last=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=shuffle_test, drop_last=False)

In [50]:
input_size = 1
hidden_size = 250
num_layers = 1
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)


criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

for epoch in range(100):
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()

In [51]:
StopWatch.start(f"prediction-{crypto}-{epoch}-{id}")
with torch.no_grad():
    for epoch in range(100):
        for i, (inputs, targets) in enumerate(test_loader):
            y_pred = model(inputs)
            if i == 0: print(inputs.shape, "/n", y_pred.shape)
            loss = criterion(y_pred, targets.view(-1, 1))
            # print("loss is", loss)
StopWatch.stop(f"prediction-{crypto}-{epoch}-{id}")

torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([32, 10, 1]) /n torch.Size([32, 1])
torch.Size([3

In [52]:
test_input = torch.empty(0)
test_target = torch.empty(0)
for i, (inputs, targets) in enumerate(test_loader):
    test_input = torch.cat((test_input, inputs), 0)
    test_target = torch.cat((test_target, targets))

model.eval()
y_pred = model(test_input)
y_pred = y_pred.squeeze(1)
criterion = nn.MSELoss()
loss = torch.sqrt(criterion(y_pred, test_target))
print("test loss is:", loss)

test loss is: tensor(0.0032, grad_fn=<SqrtBackward0>)
